In [ ]:
import torch
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import os
import time

from helpers import makedir

import model
import push
import train_and_test as tnt
from node import Node

import save
from log import create_logger
from preprocess import mean, std, preprocess_input_function, undo_preprocess_input_function
import numpy as np
import copy

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score

start = time.time()


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

data_path = "../datasets/imagenet/"
model_path = "saved_models_8protos1/"
resume_path =  model_path + "best_model_last_opt.pth"

batch_size = 50
n_protos_per_class = 8
img_size = 224

# load the data
data_path = data_path
train_dir = data_path + 'train/'
valid_dir = data_path + 'valid/'
test_dir = data_path + 'test/'
train_OOD_dir = data_path + 'OOD/train'
valid_OOD_dir = data_path + 'OOD/valid'
test_OOD_dir = data_path + 'OOD/test'
train_push_dir = train_dir
train_batch_size = batch_size
valid_batch_size = batch_size
test_batch_size = batch_size

# dataset setup

transform_test = transforms.Compose([
	transforms.Resize(256),
	transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean,std),
])


# train set
train_dataset = datasets.ImageFolder(
    train_dir,
    transform_test)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=False)
# valid set
valid_dataset = datasets.ImageFolder(
    valid_dir,
    transform_test)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=valid_batch_size, shuffle=False)    
# test set
test_dataset = datasets.ImageFolder(
    test_dir,
    transform_test)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=valid_batch_size, shuffle=False)
# OOD sets
train_OOD_dataset = datasets.ImageFolder(
    train_OOD_dir,
    transform_test)
train_OOD_loader = torch.utils.data.DataLoader(
    train_OOD_dataset, batch_size=valid_batch_size, shuffle=False)
valid_OOD_dataset = datasets.ImageFolder(
    valid_OOD_dir,
    transform_test)
valid_OOD_loader = torch.utils.data.DataLoader(
    valid_OOD_dataset, batch_size=valid_batch_size, shuffle=False)
test_OOD_dataset = datasets.ImageFolder(
    test_OOD_dir,
    transform_test)
test_OOD_loader = torch.utils.data.DataLoader(
    test_OOD_dataset, batch_size=valid_batch_size, shuffle=False)

print('training set size: {0}'.format(len(train_loader.dataset)))
print('valid set size: {0}'.format(len(valid_loader.dataset)))
print('test set size: {0}'.format(len(test_loader.dataset)))		
print('batch size: {0}'.format(train_batch_size))


# construct the tree
root = Node("root")
root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
root.add_children_to('animal',['non_primate','primate'])
root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
root.add_children_to('primate',['capuchin','gibbon','orangutan'])
root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
root.add_children_to('weapon',['assault_rifle','rifle'])
root.assign_all_descendents()
root.assign_proto_dirs()


OODroot = Node("root")
OODroot.add_children(['animal','vehicle','everyday_object','weapon',"scuba_diver"])
OODroot.add_children_to('animal',['non_primate','primate'])
OODroot.add_children_to('non_primate',['king_penguin','tree_frog','zebra'])
OODroot.add_children_to('primate',['macaque','gorilla','chimpanzee'])
OODroot.add_children_to('vehicle',['cab','forklift','tractor','mountain_bike'])
OODroot.add_children_to('everyday_object',['golf_ball','wallet','table_lamp'])
OODroot.add_children_to('weapon',['revolver','bow'])
OODroot.assign_all_descendents()
OODroot.assign_proto_dirs()


IDcoarse_names = root.children_names()
IDfine_names = os.listdir(train_dir)
IDfine_names.sort()
label2name = {i : name for (i,name) in enumerate(IDfine_names)}
IDfineLabel2coarseLabel = {label : root.children_to_labels[root.closest_descendent_for(name).name] for label, name in enumerate(IDfine_names)}    

OODcoarse_names = OODroot.children_names()
OODfine_names = os.listdir(train_OOD_dir)
OODfine_names.sort()
OODlabel2name = {i : name for (i,name) in enumerate(OODfine_names)}
OODfineLabel2coarseLabel = {label : OODroot.children_to_labels[OODroot.closest_descendent_for(name).name] for label, name in enumerate(OODfine_names)}

num_fine = len(IDfine_names)
num_coarse = len(root.children)


vgg = model.vgg16_proto(root, pretrained=True, num_prototypes_per_class=n_protos_per_class,
                        prototype_dimension = 32, img_size=img_size, resume_path = resume_path)
vgg = vgg.cuda()
vgg_multi = torch.nn.DataParallel(vgg)
class_specific=True


In [ ]:
#test_acc = tnt.test(model=vgg_multi, dataloader=valid_loader, label2name=label2name, class_specific=class_specific, log=print)

#test_acc = tnt.test(model=vgg_multi, dataloader=test_loader, label2name=label2name, class_specific=class_specific, log=print)

## Get logits for each dataset

In [ ]:

def get_X(model, dataloader, label2name, OOD = False, log=print):
    
    for node in model.module.root.nodes_with_children():
        setattr(node,"logits_all", np.ndarray(shape=(0,node.num_children())))
    
    coarse_names = model.module.root.children_names()
    num_coarse = len(coarse_names)

    fine_names = [x for x in label2name.values()]
    num_fine = len(fine_names)

    start = time.time()

    for i, (image, label) in enumerate(dataloader):
        input = image.cuda()
        target = label.cuda()

        batch_names = [label2name[y.item()] for y in label]     
        batch_size = len(target)

        batch_start = time.time()   

        cross_entropy = 0
        cluster_cost = 0
        separation_cost = 0        
        l1 = 0
        noise_cross_ent = 0
        
        #print(batch_names)
                
        with torch.no_grad():
            
            _ = model(input) 

            for node in model.module.root.nodes_with_children():
                # get names specific to children
                if not OOD:
                    children_idx = torch.tensor([name in node.descendents for name in batch_names])
                    batch_names_coarsest = [node.closest_descendent_for(name).name for name in batch_names if name in node.descendents] # size of sum(children_idx)
                    node_y = torch.tensor([node.children_to_labels[name] for name in batch_names_coarsest]).cuda() # size of sum(children_idx)
                elif OOD:
                    OODnode = OODroot.get_node(node.name)
                    children_idx = children_idx = torch.tensor([name in OODnode.descendents for name in batch_names])
                    batch_names_coarsest = [OODnode.closest_descendent_for(name).name for name in batch_names if name in OODnode.descendents] # size of sum(children_idx)
                    node_y = torch.tensor([OODnode.children_to_labels[name] for name in batch_names_coarsest]).cuda() # size of sum(children_idx)
                    #print(batch_names_coarsest)
                
                node_logits = node.logits[children_idx]            
                node_logits = node_logits.detach().cpu().numpy()
                
                if len(node_y) > 0:
                    node.logits_all = np.concatenate((node.logits_all,node_logits))
                                        
        del input
        del target, node_y
        
        batch_end = time.time()
        

        
    end = time.time()

    log('time: {0:.2f}'.format(end -  start))

    return None   


In [ ]:
_ = get_X(vgg_multi, valid_loader, label2name)

for node in vgg.root.nodes_with_children():
    print(node.name, node.logits_all.shape)


## Here's the plan

train
IDtrain_subset : OOD1250classes1

valid
IDvalid_subset : OOD50classes1

test
IDtest : OOD50classes2

For each of the furthest parents, cycle through possible splits at each set of lowest parents


In [ ]:
furthest_parents = ['non_primate','primate','vehicle','everday_object','weapon']

## ID logits

In [ ]:
np.random.seed(2019)

# for balanced data, need to subset the ID data each time, since when fitting the OOD classes get split between train and test
balance1_idx = np.random.choice(np.arange(3750),2500,replace=False)
balance2_idx = np.random.choice(np.arange(2500),1250,replace=False)

# train data
_ = get_X(vgg_multi, train_loader, label2name)

train_np_X = vgg.root.get_node("non_primate").logits_all[balance1_idx]

train_p_X = vgg.root.get_node("primate").logits_all[balance1_idx]

train_v_X = vgg.root.get_node("vehicle").logits_all[balance1_idx]

train_e_X = vgg.root.get_node("everyday_object").logits_all[balance1_idx]

train_w_X = vgg.root.get_node("weapon").logits_all[balance2_idx]

print(train_np_X.shape)
print(train_w_X.shape)

balance1_idx = np.random.choice(np.arange(150),100,replace=False)
balance2_idx = np.random.choice(np.arange(100),50,replace=False)

# valid data
_ = get_X(vgg_multi, valid_loader, label2name)

valid_np_X = vgg.root.get_node("non_primate").logits_all[balance1_idx]

valid_p_X = vgg.root.get_node("primate").logits_all[balance1_idx]

valid_v_X = vgg.root.get_node("vehicle").logits_all[balance1_idx]

valid_e_X = vgg.root.get_node("everyday_object").logits_all[balance1_idx]

valid_w_X = vgg.root.get_node("weapon").logits_all[balance2_idx]

print(valid_np_X.shape)
print(valid_w_X.shape)

balance1_idx = np.random.choice(np.arange(150),50,replace=False)
balance2_idx = np.random.choice(np.arange(100),50,replace=False)

# test data
_ = get_X(vgg_multi, test_loader, label2name)

test_np_X = vgg.root.get_node("non_primate").logits_all[balance1_idx]

test_p_X = vgg.root.get_node("primate").logits_all[balance1_idx]

test_v_X = vgg.root.get_node("vehicle").logits_all[balance1_idx]

test_e_X = vgg.root.get_node("everyday_object").logits_all[balance1_idx]

test_w_X = vgg.root.get_node("weapon").logits_all[balance2_idx]

print(test_np_X.shape)
print(test_w_X.shape)


## Novel logits

In [ ]:
# make train
_ = get_X(vgg_multi, train_OOD_loader, OODlabel2name,  OOD = True)

train_np_logits = vgg.root.get_node("non_primate").logits_all
train_p_logits = vgg.root.get_node("primate").logits_all
train_v_logits = vgg.root.get_node("vehicle").logits_all
train_e_logits = vgg.root.get_node("everyday_object").logits_all
train_w_logits = vgg.root.get_node("weapon").logits_all

print(train_np_logits.shape)
print(train_w_logits.shape)

# make valid
_ = get_X(vgg_multi, valid_OOD_loader, OODlabel2name, OOD = True)

valid_np_logits = vgg.root.get_node("non_primate").logits_all
valid_p_logits = vgg.root.get_node("primate").logits_all
valid_v_logits = vgg.root.get_node("vehicle").logits_all
valid_e_logits = vgg.root.get_node("everyday_object").logits_all
valid_w_logits = vgg.root.get_node("weapon").logits_all

print(valid_np_logits.shape)
print(valid_w_logits.shape)

# make test
_ = get_X(vgg_multi, test_OOD_loader, OODlabel2name, OOD = True)

test_np_logits = vgg.root.get_node("non_primate").logits_all
test_p_logits = vgg.root.get_node("primate").logits_all
test_v_logits = vgg.root.get_node("vehicle").logits_all
test_e_logits = vgg.root.get_node("everyday_object").logits_all
test_w_logits = vgg.root.get_node("weapon").logits_all

print(test_np_logits.shape)
print(test_w_logits.shape)

In [ ]:
trial = 1
test_idx = np.arange(1250*trial,1250*(trial+1)) # these never actually index anything
train_valid_idx = np.setdiff1d(np.arange(3750),test_idx)

# train

full_train_np_X = np.concatenate((train_np_X,train_np_logits[train_valid_idx]))
full_train_np_Y = np.concatenate((np.zeros(2500),np.ones(2500)))

full_train_p_X = np.concatenate((train_p_X,train_p_logits[train_valid_idx]))
full_train_p_Y = np.concatenate((np.zeros(2500),np.ones(2500)))

full_train_v_X = np.concatenate((train_v_X,train_v_logits[train_valid_idx]))
full_train_v_Y = np.concatenate((np.zeros(2500),np.ones(2500)))

full_train_e_X = np.concatenate((train_e_X,train_e_logits[train_valid_idx]))
full_train_e_Y = np.concatenate((np.zeros(2500),np.ones(2500)))

train_valid_idx = np.setdiff1d(np.arange(2500),test_idx)
full_train_w_X = np.concatenate((train_w_X,train_w_logits[train_valid_idx]))
full_train_w_Y = np.concatenate((np.zeros(1250),np.ones(1250)))

# valid

test_idx = np.arange(50*trial,50*(trial+1))
train_valid_idx = np.setdiff1d(np.arange(150),test_idx)


full_valid_np_X = np.concatenate((valid_np_X,valid_np_logits[train_valid_idx]))
full_valid_np_Y = np.concatenate((np.zeros(100),np.ones(100)))

full_valid_p_X = np.concatenate((valid_p_X,valid_p_logits[train_valid_idx]))
full_valid_p_Y = np.concatenate((np.zeros(100),np.ones(100)))

full_valid_v_X = np.concatenate((valid_v_X,valid_v_logits[train_valid_idx]))
full_valid_v_Y = np.concatenate((np.zeros(100),np.ones(100)))

full_valid_e_X = np.concatenate((valid_e_X,valid_e_logits[train_valid_idx]))
full_valid_e_Y = np.concatenate((np.zeros(100),np.ones(100)))

train_valid_idx = np.setdiff1d(np.arange(100),test_idx)
full_valid_w_X = np.concatenate((valid_w_X,valid_w_logits[train_valid_idx]))
full_valid_w_Y = np.concatenate((np.zeros(50),np.ones(50)))

# test

full_test_np_X = np.concatenate((test_np_X,test_np_logits[test_idx]))
full_test_np_Y = np.concatenate((np.zeros(50),np.ones(50)))

full_test_p_X = np.concatenate((test_p_X,test_p_logits[test_idx]))
full_test_p_Y = np.concatenate((np.zeros(50),np.ones(50)))

full_test_v_X = np.concatenate((test_v_X,test_v_logits[test_idx]))
full_test_v_Y = np.concatenate((np.zeros(50),np.ones(50)))

full_test_e_X = np.concatenate((test_e_X,test_e_logits[test_idx]))
full_test_e_Y = np.concatenate((np.zeros(50),np.ones(50)))

full_test_w_X = np.concatenate((test_w_X,test_w_logits[test_idx]))
full_test_w_Y = np.concatenate((np.zeros(50),np.ones(50)))


In [ ]:
def score_thresh(thresh,X,Y):
    Yhat = 1 * (X > thresh)
    return np.mean(Yhat == Y)

def line_search(X,Y,probs=False):        
    thresholds = np.arange(np.min(X),np.max(X), (np.max(X) - np.min(X)) / 150)
    thresh2acc = {}
    for thresh in thresholds:
        thresh2acc[thresh] = score_thresh(thresh,X,Y)
    return np.array([x for x in thresh2acc.items()])

def full_line_search(train_X,train_Y,test_X,test_Y,probs=False):
    if probs:
        train_Xprime = torch.nn.functional.softmax(torch.tensor(train_X),1).numpy()
        test_Xprime = torch.nn.functional.softmax(torch.tensor(test_X),1).numpy()
        train_Xprime = np.max(train_Xprime,1)
        test_Xprime = np.max(test_Xprime,1)
    else:
        train_Xprime = np.max(train_X,1)
        test_Xprime = np.max(test_X,1)    
    
    train = line_search(train_Xprime,train_Y,probs=probs)
    
    best_id = np.argmax(train[:,1])
    best_thresh = train[:,0][best_id]    
    
    train_acc = score_thresh(best_thresh,train_Xprime,train_Y)
    test_acc = score_thresh(best_thresh,test_Xprime,test_Y)
    
    
    return best_thresh, test_acc, test_acc


from sklearn.linear_model import LogisticRegression

def lr_line_search(X,Y,probs=False):
    
    model = LogisticRegression(C=1e6)            
    model.fit(X,Y)    
    
    return model
    
def full_lr_line_search(train_X,train_Y,test_X,test_Y,probs=False):
    if probs:
        train_Xprime = torch.nn.functional.softmax(torch.tensor(train_X),1).numpy()
        test_Xprime = torch.nn.functional.softmax(torch.tensor(test_X),1).numpy()
        #train_Xprime = np.max(train_Xprime,1).reshape(-1,1)
        #test_Xprime = np.max(test_Xprime,1).reshape(-1,1)
    else:
        train_Xprime = train_X#np.max(train_X,1).reshape(-1,1)#train_X 
        test_Xprime = test_X#np.max(test_X,1).reshape(-1,1) #test_X 
    
    model = lr_line_search(train_Xprime,train_Y,probs=probs)
            
    train_acc = model.score(train_Xprime,train_Y)
    test_acc = model.score(test_Xprime,test_Y)    
    
    return 0, train_acc, test_acc
    

## Non primates

In [ ]:
np.random.seed(2019)

gs = [1] # using linear kernel now
cs = [.001,.01,.1,.5,1,10]

node = OODroot.get_node("non_primate")
print("OOD non primates: ", node.children_names())

trial_accs = np.zeros(node.num_children())

logit_thresh_accs = np.zeros(node.num_children())
probs_thresh_accs = np.zeros(node.num_children())

for trial in range(node.num_children()):
    
    print("\ntrial: ", trial)
    print("held out class: %s \n" % node.children[trial].name)
    
    train_accs = np.ndarray(shape=(len(gs),len(cs)))
    valid_accs = np.ndarray(shape=(len(gs),len(cs)))
    
    # get data
    test_idx = np.arange(1250*trial,1250*(trial+1)) # these never actually index anything
    train_valid_idx = np.setdiff1d(np.arange(3750),test_idx)
    
    full_train_np_X = np.concatenate((train_np_X,train_np_logits[train_valid_idx]))
    full_train_np_Y = np.concatenate((np.zeros(2500),np.ones(2500)))

    test_idx = np.arange(50*trial,50*(trial+1))
    train_valid_idx = np.setdiff1d(np.arange(150),test_idx)

    full_valid_np_X = np.concatenate((valid_np_X,valid_np_logits[train_valid_idx]))
    full_valid_np_Y = np.concatenate((np.zeros(100),np.ones(100)))    
    
    full_test_np_X = np.concatenate((test_np_X,test_np_logits[test_idx]))
    full_test_np_Y = np.concatenate((np.zeros(50),np.ones(50)))
    
    
    # thresholding
    logit_thresh, _, logit_thresh_acc = full_lr_line_search(full_train_np_X,full_train_np_Y,
                                                      full_test_np_X,full_test_np_Y,probs=False)  
    
    logit_thresh_accs[trial] = logit_thresh_acc
    
    probs_thresh, _, probs_thresh_acc = full_lr_line_search(full_train_np_X,full_train_np_Y,
                                                      full_test_np_X,full_test_np_Y,probs=True)    
    probs_thresh_accs[trial] = probs_thresh_acc
    
    
    # SVM cross validation fitting
    for g in range(len(gs)):
        for c in range(len(cs)): 

#             print("iter: %d / %d \t starting g: %.3f and c: %.3f" % (g*len(cs) + c, len(gs) * len(cs), gs[g],cs[c]))

            SVM = SVC(kernel='linear',gamma=gs[g],C=cs[c])
            SVM.fit(full_train_np_X,full_train_np_Y)

            yhat = SVM.predict(full_train_np_X)
            ytrue = full_train_np_Y
            conf_mat = confusion_matrix(ytrue,yhat)

            train_acc = SVM.score(full_train_np_X,full_train_np_Y)
            valid_acc = SVM.score(full_valid_np_X,full_valid_np_Y)

            train_accs[g,c] = train_acc
            valid_accs[g,c] = valid_acc

    row_id = valid_accs.argmax() // len(cs)
    col_id = valid_accs.argmax() - (row_id+1)*len(cs)
    best_g = gs[row_id]
    best_c = cs[col_id]
    
    SVM = SVC(kernel='linear',gamma=best_g,C=best_c)
    SVM.fit(full_train_np_X,full_train_np_Y)

    test_acc = SVM.score(full_test_np_X,full_test_np_Y)
    
    trial_accs[trial] = test_acc
    
    print("\n valid accs \n")
    print(valid_accs)
    
    print("best g: ", best_g)
    print("best c: ", best_c)
        
    print("test acc: ",test_acc)

    

In [ ]:
print("SVM accs")
print(trial_accs)
print("mean acc: ",np.mean(trial_accs))
np_svm_mean = np.mean(trial_accs)

print("\nlogit thresh accs")
print(logit_thresh_accs)
print("mean acc: ",np.mean(logit_thresh_accs))
np_logit_mean = np.mean(logit_thresh_accs)

print("\nprobs thresh accs")
print(probs_thresh_accs)
print("mean acc: ",np.mean(probs_thresh_accs))
np_prob_mean = np.mean(probs_thresh_accs)


## primates

In [ ]:
np.random.seed(2019)

gs = [1] # using linear kernel now
cs = [.001,.01,.1,.5,1,10]


node = OODroot.get_node("primate")
print("OOD primates: ", node.children_names())

trial_accs = np.zeros(node.num_children())

logit_thresh_accs = np.zeros(node.num_children())
probs_thresh_accs = np.zeros(node.num_children())

for trial in range(node.num_children()):
    
    print("\ntrial: ", trial)
    print("held out class: %s \n" % node.children[trial].name)
    
    train_accs = np.ndarray(shape=(len(gs),len(cs)))
    valid_accs = np.ndarray(shape=(len(gs),len(cs)))
    
    # get data
    test_idx = np.arange(1250*trial,1250*(trial+1)) # these never actually index anything
    train_valid_idx = np.setdiff1d(np.arange(3750),test_idx)
    
    full_train_p_X = np.concatenate((train_p_X,train_p_logits[train_valid_idx]))
    full_train_p_Y = np.concatenate((np.zeros(2500),np.ones(2500)))

    test_idx = np.arange(50*trial,50*(trial+1))
    train_valid_idx = np.setdiff1d(np.arange(150),test_idx)

    full_valid_p_X = np.concatenate((valid_p_X,valid_p_logits[train_valid_idx]))
    full_valid_p_Y = np.concatenate((np.zeros(100),np.ones(100)))  
    
    full_test_p_X = np.concatenate((test_p_X,test_p_logits[test_idx]))
    full_test_p_Y = np.concatenate((np.zeros(50),np.ones(50)))
            
    # thresholding
    logit_thresh, _, logit_thresh_acc = full_lr_line_search(full_train_p_X,full_train_p_Y,
                                                      full_test_p_X,full_test_p_Y,probs=False)  
    
    logit_thresh_accs[trial] = logit_thresh_acc
    
    probs_thresh, _, probs_thresh_acc = full_lr_line_search(full_train_p_X,full_train_p_Y,
                                                      full_test_p_X,full_test_p_Y,probs=True)    
    probs_thresh_accs[trial] = probs_thresh_acc
    
    
    # SVM cross validation fitting
    for g in range(len(gs)):
        for c in range(len(cs)): 

#             print("iter: %d / %d \t starting g: %.3f and c: %.3f" % (g*len(cs) + c, len(gs) * len(cs), gs[g],cs[c]))

            SVM = SVC(kernel='linear',gamma=gs[g],C=cs[c])
            SVM.fit(full_train_p_X,full_train_p_Y)

            yhat = SVM.predict(full_train_p_X)
            ytrue = full_train_p_Y
            conf_mat = confusion_matrix(ytrue,yhat)

    #         print("\nconf mat \n")
    #         print(np.stack([[np.round(x/np.sum(row),2) for x in row] for row in conf_mat]))        

            train_acc = SVM.score(full_train_p_X,full_train_p_Y)
            valid_acc = SVM.score(full_valid_p_X,full_valid_p_Y)

    #         print("train acc: %.2f " % train_acc)
    #         print("valid acc: %.2f " % valid_acc)

            train_accs[g,c] = train_acc
            valid_accs[g,c] = valid_acc

    row_id = valid_accs.argmax() // len(cs)
    col_id = valid_accs.argmax() - (row_id+1)*len(cs)
    best_g = gs[row_id]
    best_c = cs[col_id]
    
    SVM = SVC(kernel='linear',gamma=best_g,C=best_c)
    SVM.fit(full_train_p_X,full_train_p_Y)

    test_acc = SVM.score(full_test_p_X,full_test_p_Y)
    
    trial_accs[trial] = test_acc
    
    print("\n valid accs \n")
    print(valid_accs)
    
    print("best g: ", best_g)
    print("best c: ", best_c)
        
    print("test acc: ",test_acc)

    

In [ ]:
print("SVM accs")
print(trial_accs)
print("mean acc: ",np.mean(trial_accs))
p_svm_mean = np.mean(trial_accs)

print("\nlogit thresh accs")
print(logit_thresh_accs)
print("mean acc: ",np.mean(logit_thresh_accs))
p_logit_mean = np.mean(logit_thresh_accs)

print("\nprobs thresh accs")
print(probs_thresh_accs)
print("mean acc: ",np.mean(probs_thresh_accs))
p_prob_mean = np.mean(probs_thresh_accs)


## vehicles

In [ ]:
np.random.seed(2019)

gs = [1] # using linear kernel now
cs = [.001,.01,.1,.5,1,10]

node = OODroot.get_node("vehicle")
print("OOD vehicles: ", node.children_names())

trial_accs = np.zeros(node.num_children())

logit_thresh_accs = np.zeros(node.num_children())
probs_thresh_accs = np.zeros(node.num_children())


for trial in range(node.num_children()):
    
    print("\ntrial: ", trial)
    print("held out class: %s \n" % node.children[trial].name)
    
    train_accs = np.ndarray(shape=(len(gs),len(cs)))
    valid_accs = np.ndarray(shape=(len(gs),len(cs)))
    
    # get data
    test_idx = np.arange(1250*trial,1250*(trial+1)) # these never actually index anything
    train_valid_idx = np.setdiff1d(np.arange(5000),test_idx)
    
    full_train_v_X = np.concatenate((train_v_X,train_v_logits[train_valid_idx]))
    full_train_v_Y = np.concatenate((np.zeros(2500),np.ones(3750))) # data was not balanced here

    print(trial)
    test_idx = np.arange(50*trial,50*(trial+1))
    train_valid_idx = np.setdiff1d(np.arange(200),test_idx)

    full_valid_v_X = np.concatenate((valid_v_X,valid_v_logits[train_valid_idx]))
    full_valid_v_Y = np.concatenate((np.zeros(100),np.ones(150)))    

    full_test_v_X = np.concatenate((test_v_X,test_v_logits[test_idx]))
    full_test_v_Y = np.concatenate((np.zeros(50),np.ones(50)))
    
    # thresholding
    logit_thresh, _, logit_thresh_acc = full_lr_line_search(full_train_v_X,full_train_v_Y,
                                                      full_test_v_X,full_test_v_Y,probs=False)  
    
    logit_thresh_accs[trial] = logit_thresh_acc
    
    probs_thresh, _, probs_thresh_acc = full_lr_line_search(full_train_v_X,full_train_v_Y,
                                                      full_test_v_X,full_test_v_Y,probs=True)    
    probs_thresh_accs[trial] = probs_thresh_acc
    
    
    # SVM cross validation fitting
    for g in range(len(gs)):
        for c in range(len(cs)): 

#             print("iter: %d / %d \t starting g: %.3f and c: %.3f" % (g*len(cs) + c, len(gs) * len(cs), gs[g],cs[c]))

            SVM = SVC(kernel='linear',gamma=gs[g],C=cs[c])
            SVM.fit(full_train_v_X,full_train_v_Y)

            yhat = SVM.predict(full_train_v_X)
            ytrue = full_train_v_Y
            conf_mat = confusion_matrix(ytrue,yhat)

    #         print("\nconf mat \n")
    #         print(np.stack([[np.round(x/np.sum(row),2) for x in row] for row in conf_mat]))        

            train_acc = SVM.score(full_train_v_X,full_train_v_Y)
            valid_acc = SVM.score(full_valid_v_X,full_valid_v_Y)

    #         print("train acc: %.2f " % train_acc)
    #         print("valid acc: %.2f " % valid_acc)

            train_accs[g,c] = train_acc
            valid_accs[g,c] = valid_acc

    row_id = valid_accs.argmax() // len(cs)
    col_id = valid_accs.argmax() - (row_id+1)*len(cs)
    best_g = gs[row_id]
    best_c = cs[col_id]
    
    SVM = SVC(kernel='linear',gamma=best_g,C=best_c)
    SVM.fit(full_train_v_X,full_train_v_Y)

    test_acc = SVM.score(full_test_v_X,full_test_v_Y)
    
    trial_accs[trial] = test_acc
    
    print("\n valid accs \n")
    print(valid_accs)
    
    print("best g: ", best_g)
    print("best c: ", best_c)
        
    print("test acc: ",test_acc)

    

In [ ]:
print("SVM accs")
print(trial_accs)
print("mean acc: ",np.mean(trial_accs))
v_svm_mean = np.mean(trial_accs)

print("\nlogit thresh accs")
print(logit_thresh_accs)
print("mean acc: ",np.mean(logit_thresh_accs))
v_logit_mean = np.mean(logit_thresh_accs)

print("\nprobs thresh accs")
print(probs_thresh_accs)
print("mean acc: ",np.mean(probs_thresh_accs))
v_prob_mean = np.mean(probs_thresh_accs)


## everyday object

In [ ]:
np.random.seed(2019)

gs = [1] # using linear kernel now
cs = [.001,.01,.1,.5,1,10]

node = OODroot.get_node("everyday_object")
print("OOD everyday objects: ", node.children_names())

trial_accs = np.zeros(node.num_children())

logit_thresh_accs = np.zeros(node.num_children())
probs_thresh_accs = np.zeros(node.num_children())


for trial in range(node.num_children()):
    
    print("\ntrial: ", trial)
    print("held out class: %s \n" % node.children[trial].name)
    
    train_accs = np.ndarray(shape=(len(gs),len(cs)))
    valid_accs = np.ndarray(shape=(len(gs),len(cs)))
    
    # get data
    test_idx = np.arange(1250*trial,1250*(trial+1)) # these never actually index anything
    train_valid_idx = np.setdiff1d(np.arange(3750),test_idx)
    
    full_train_e_X = np.concatenate((train_e_X,train_e_logits[train_valid_idx]))
    full_train_e_Y = np.concatenate((np.zeros(2500),np.ones(2500)))

    test_idx = np.arange(50*trial,50*(trial+1))
    train_valid_idx = np.setdiff1d(np.arange(150),test_idx)

    full_valid_e_X = np.concatenate((valid_e_X,valid_e_logits[train_valid_idx]))
    full_valid_e_Y = np.concatenate((np.zeros(100),np.ones(100)))    
    
    full_test_e_X = np.concatenate((test_e_X,test_e_logits[test_idx]))
    full_test_e_Y = np.concatenate((np.zeros(50),np.ones(50)))

    
    # thresholding
    logit_thresh, _, logit_thresh_acc = full_lr_line_search(full_train_e_X,full_train_e_Y,
                                                      full_test_e_X,full_test_e_Y,probs=False)  
    
    logit_thresh_accs[trial] = logit_thresh_acc
    
    probs_thresh, _, probs_thresh_acc = full_lr_line_search(full_train_e_X,full_train_e_Y,
                                                      full_test_e_X,full_test_e_Y,probs=True)    
    probs_thresh_accs[trial] = probs_thresh_acc
    
    
    # SVM cross validation fitting
    for g in range(len(gs)):
        for c in range(len(cs)): 

#             print("iter: %d / %d \t starting g: %.3f and c: %.3f" % (g*len(cs) + c, len(gs) * len(cs), gs[g],cs[c]))

            SVM = SVC(kernel='linear',gamma=gs[g],C=cs[c])
            SVM.fit(full_train_e_X,full_train_e_Y)

            yhat = SVM.predict(full_train_e_X)
            ytrue = full_train_e_Y
            conf_mat = confusion_matrix(ytrue,yhat)

    #         print("\nconf mat \n")
    #         print(np.stack([[np.round(x/np.sum(row),2) for x in row] for row in conf_mat]))        

            train_acc = SVM.score(full_train_e_X,full_train_e_Y)
            valid_acc = SVM.score(full_valid_e_X,full_valid_e_Y)

    #         print("train acc: %.2f " % train_acc)
    #         print("valid acc: %.2f " % valid_acc)

            train_accs[g,c] = train_acc
            valid_accs[g,c] = valid_acc

    row_id = valid_accs.argmax() // len(cs)
    col_id = valid_accs.argmax() - (row_id+1)*len(cs)
    best_g = gs[row_id]
    best_c = cs[col_id]
    
    SVM = SVC(kernel='linear',gamma=best_g,C=best_c)
    SVM.fit(full_train_e_X,full_train_e_Y)

    test_acc = SVM.score(full_test_e_X,full_test_e_Y)
    
    trial_accs[trial] = test_acc
    
    print("\n valid accs \n")
    print(valid_accs)
    
    print("best g: ", best_g)
    print("best c: ", best_c)
        
    print("test acc: ",test_acc)

    

In [ ]:
print("SVM accs")
print(trial_accs)
print("mean acc: ",np.mean(trial_accs))
e_svm_mean = np.mean(trial_accs)

print("\nlogit thresh accs")
print(logit_thresh_accs)
print("mean acc: ",np.mean(logit_thresh_accs))
e_logit_mean = np.mean(logit_thresh_accs)

print("\nprobs thresh accs")
print(probs_thresh_accs)
print("mean acc: ",np.mean(probs_thresh_accs))
e_prob_mean = np.mean(probs_thresh_accs)


## weapon

In [ ]:
np.random.seed(2019)

gs = [1] # using linear kernel now
cs = [.001,.01,.1,.5,1,10]

node = OODroot.get_node("weapon")
print("OOD weapons: ", node.children_names())

trial_accs = np.zeros(node.num_children())

logit_thresh_accs = np.zeros(node.num_children())
probs_thresh_accs = np.zeros(node.num_children())


for trial in range(node.num_children()):
    
    print("\ntrial: ", trial)
    print("held out class: %s \n" % node.children[trial].name)
    
    train_accs = np.ndarray(shape=(len(gs),len(cs)))
    valid_accs = np.ndarray(shape=(len(gs),len(cs)))
    
    # get data
    test_idx = np.arange(1250*trial,1250*(trial+1)) # these never actually index anything
    train_valid_idx = np.setdiff1d(np.arange(2500),test_idx)
    
    full_train_w_X = np.concatenate((train_w_X,train_w_logits[train_valid_idx]))
    full_train_w_Y = np.concatenate((np.zeros(1250),np.ones(1250)))

    test_idx = np.arange(50*trial,50*(trial+1))
    train_valid_idx = np.setdiff1d(np.arange(100),test_idx)

    full_valid_w_X = np.concatenate((valid_w_X,valid_w_logits[train_valid_idx]))
    full_valid_w_Y = np.concatenate((np.zeros(50),np.ones(50)))    
    
    full_test_w_X = np.concatenate((test_w_X,test_w_logits[test_idx]))
    full_test_w_Y = np.concatenate((np.zeros(50),np.ones(50)))
    
    # thresholding
    logit_thresh, _, logit_thresh_acc = full_lr_line_search(full_train_w_X,full_train_w_Y,
                                                      full_test_w_X,full_test_w_Y,probs=False)  
    
    logit_thresh_accs[trial] = logit_thresh_acc
    
    probs_thresh, _, probs_thresh_acc = full_lr_line_search(full_train_w_X,full_train_w_Y,
                                                      full_test_w_X,full_test_w_Y,probs=True)    
    probs_thresh_accs[trial] = probs_thresh_acc
    
    
    # SVM cross validation fitting
    for g in range(len(gs)):
        for c in range(len(cs)): 

#             print("iter: %d / %d \t starting g: %.3f and c: %.3f" % (g*len(cs) + c, len(gs) * len(cs), gs[g],cs[c]))

            SVM = SVC(kernel='linear',gamma=gs[g],C=cs[c])
            SVM.fit(full_train_w_X,full_train_w_Y)

            yhat = SVM.predict(full_train_w_X)
            ytrue = full_train_w_Y
            conf_mat = confusion_matrix(ytrue,yhat)

            train_acc = SVM.score(full_train_w_X,full_train_w_Y)
            valid_acc = SVM.score(full_valid_w_X,full_valid_w_Y)

            train_accs[g,c] = train_acc
            valid_accs[g,c] = valid_acc

    row_id = valid_accs.argmax() // len(cs)
    col_id = valid_accs.argmax() - (row_id+1)*len(cs)
    best_g = gs[row_id]
    best_c = cs[col_id]
    
    SVM = SVC(kernel='linear',gamma=best_g,C=best_c)
    SVM.fit(full_train_w_X,full_train_w_Y)

    test_acc = SVM.score(full_test_w_X,full_test_w_Y)
    
    trial_accs[trial] = test_acc
    
    print("\n valid accs \n")
    print(valid_accs)
    
    print("best g: ", best_g)
    print("best c: ", best_c)
        
    print("test acc: ",test_acc)

    

In [ ]:
print("SVM accs")
print(trial_accs)
print("mean acc: ",np.mean(trial_accs))
w_svm_mean = np.mean(trial_accs)

print("\nlogit thresh accs")
print(logit_thresh_accs)
print("mean acc: ",np.mean(logit_thresh_accs))
w_logit_mean = np.mean(logit_thresh_accs)

print("\nprobs thresh accs")
print(probs_thresh_accs)
print("mean acc: ",np.mean(probs_thresh_accs))
w_prob_mean = np.mean(probs_thresh_accs)


## Totals

In [ ]:
# means across class

svm_mean = np.mean([np_svm_mean,p_svm_mean,w_svm_mean,e_svm_mean,v_svm_mean])
print("svm mean: ", svm_mean)
logit_mean = np.mean([np_logit_mean,p_logit_mean,w_logit_mean,e_logit_mean,v_logit_mean])
print("logit mean: ", logit_mean)
prob_mean = np.mean([np_prob_mean,p_prob_mean,w_prob_mean,e_prob_mean,v_prob_mean])
print("prob mean: ", prob_mean)


## Pick a specific instance


In [ ]:

trial = 1

node = OODroot.get_node("weapon")
    
print("\ntrial: ", trial)
print("held out class: %s \n" % node.children[trial].name)

# get data
test_idx = np.arange(1250*trial,1250*(trial+1)) # these never actually index anything
train_valid_idx = np.setdiff1d(np.arange(2500),test_idx)

full_train_w_X = np.concatenate((train_w_X,train_w_logits[train_valid_idx]))
full_train_w_Y = np.concatenate((np.zeros(1250),np.ones(1250)))

test_idx = np.arange(50*trial,50*(trial+1))

full_test_w_X = np.concatenate((test_w_X,test_w_logits[test_idx]))
full_test_w_Y = np.concatenate((np.zeros(50),np.ones(50)))

model = lr_line_search(full_train_w_X,full_train_w_Y)

# conf mat
yhat = model.predict(full_test_w_X)
ytrue = full_test_w_Y
conf_mat = confusion_matrix(ytrue,yhat)
print("\nconf mat \n")
print(np.stack([[np.round(x/np.sum(row),2) for x in row] for row in conf_mat]))        


yhat = model.predict_proba(test_w_logits[test_idx])[:,1]
correct = yhat > 1/2

test_acc = np.mean(correct)
print("test acc: ",test_acc)

probs = np.array([np.round(x,2) for x in yhat])

print("\nidx of predicted novel:")
yhat_pos = np.where(yhat>1/2)[0]
zipped = [x for x in zip(probs[yhat_pos],yhat_pos)]

for z in zipped:
    print(z)
    

In [ ]:

for trial in [0,1]:#,2,3]:
    #trial = 1

    node = OODroot.get_node("vehicle")

    print("\ntrial: ", trial)
    print("held out class: %s \n" % node.children[trial].name)

    # get data
    test_idx = np.arange(1250*trial,1250*(trial+1)) # these never actually index anything
    train_valid_idx = np.setdiff1d(np.arange(5000),test_idx)

    full_train_v_X = np.concatenate((train_v_X,train_v_logits[train_valid_idx]))
    full_train_v_Y = np.concatenate((np.zeros(2500),np.ones(3750)))
    
    test_idx = np.arange(50*trial,50*(trial+1))
    
    full_test_v_X = np.concatenate((test_v_X,test_v_logits[test_idx]))
    full_test_v_Y = np.concatenate((np.zeros(50),np.ones(50)))

    model = lr_line_search(full_train_v_X,full_train_v_Y)

    # conf mat
    yhat = model.predict(full_test_v_X)
    ytrue = full_test_v_Y
    conf_mat = confusion_matrix(ytrue,yhat)
    print("\nconf mat \n")
    print(np.stack([[np.round(x/np.sum(row),2) for x in row] for row in conf_mat]))        
    
    
    yhat = model.predict_proba(test_v_logits[test_idx])[:,1]
    correct = yhat > 1/2

    test_acc = np.mean(correct)
    print("test acc: ",test_acc)

    probs = np.array([np.round(x,2) for x in yhat])

    print("\nidx of predicted novel:")
    yhat_pos = np.where(yhat>1/2)[0]
    zipped = [x for x in zip(probs[yhat_pos],yhat_pos)]

    for z in zipped:
        print(z)

        

In [ ]:
end = time.time()

print('time: \t{0:.2f}'.format(end -  start))

In [ ]:
# means across class

svm_mean = np.mean([np_svm_mean,p_svm_mean,w_svm_mean,e_svm_mean,v_svm_mean])
print("svm mean: ", svm_mean)
logit_mean = np.mean([np_logit_mean,p_logit_mean,w_logit_mean,e_logit_mean,v_logit_mean])
print("logit mean: ", logit_mean)
prob_mean = np.mean([np_prob_mean,p_prob_mean,w_prob_mean,e_prob_mean,v_prob_mean])
print("prob mean: ", prob_mean)
